In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newninaprodb4/ninaprodb44test.pkl
/kaggle/input/newninaprodb4/ninaprodb44train.pkl


In [2]:
import numpy as np
import os
from tensorflow.keras.layers import TimeDistributed, Conv1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation

# ... (other code) ...


import random
#from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import math

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras import layers, Sequential, optimizers, Input, Model

**WORKED CODE HERE**

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import gc
import random
import os

import matplotlib.pyplot as plt
import json

In [6]:
epochs = 64
learning_rate = 1e-4
batch_size = 32
method = "default"
dataset_type = 1

In [7]:
pip install PyWavelets


Note: you may need to restart the kernel to use updated packages.


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pywt  # Added import for wavelet denoising
import scipy.signal as signal  # Added import for power line noise and low pass filtering
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pywt
import scipy.signal as signal
import tensorflow as tf
class Nina1Dataset(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.scaler = StandardScaler()
        self.scaler.fit(np.concatenate(self.dataframe['emg'].tolist()))
        
        
    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.dataframe[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_input_data = []
        batch_labels = []

        for i, target_row in batch_data.iterrows():
            data = target_row['emg'][:10000]

            # Zero-Padding
            if len(data) < 10000:
                data = np.concatenate((data, np.zeros((10000 - len(data), 12))), axis=0)

            
            
            fs = 2000  # Assuming sampling frequency of 2000 Hz
            f0 = 50  # Power line frequency
            Q = 30  # Quality factor
            w0 = f0 / (fs / 2)
            b, a = signal.iirnotch(w0, Q)
            data = signal.lfilter(b, a, data, axis=0)

                    # Low Pass Filtering with cutoff frequency fc = 500 Hz
            fc = 500  # Cutoff frequency
            b, a = signal.butter(4, fc / (fs / 2), 'low')
            data = signal.lfilter(b, a, data, axis=0)


            coeffs = pywt.wavedec(data, 'sym8')  
            coeffs[1:] = (pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:])
            data = pywt.waverec(coeffs, 'sym8')


                # Wavelet Denoising




                    # Standardize the data
            data = self.scaler.transform(data)
                #data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

                    # Division data by time-segment (channel-wise)
            input_data = data.reshape((25, 400, 12))
            label = target_row['stimulus']
            batch_input_data.append(input_data)
            batch_labels.append(label)

        # Check if the batch size is smaller than the desired batch_size
        if len(batch_data) < self.batch_size:
            # Create a dummy batch with all elements set to zero
            dummy_input_data = np.zeros((self.batch_size,) + input_data.shape, dtype=np.float32)
            dummy_labels = np.zeros((self.batch_size,), dtype=np.int32)
            dummy_input_data[:len(batch_input_data)] = np.array(batch_input_data)
            dummy_labels[:len(batch_labels)] = np.array(batch_labels)
            dummy_labels = to_categorical(dummy_labels, num_classes=52)  # Convert labels to one-hot encoding

            return dummy_input_data, dummy_labels

        batch_labels = to_categorical(batch_labels, num_classes=52)  # Convert labels to one-hot encoding

        return np.array(batch_input_data), np.array(batch_labels)

# Parameters

batch_size = 32
train_dir = '/kaggle/input/newninaprodb4/ninaprodb44train.pkl'
test_dir = '/kaggle/input/newninaprodb4/ninaprodb44test.pkl'

# Set up dataset
train = pd.read_pickle(train_dir)
eval_data = pd.read_pickle(test_dir)

# Load train data
train_data = pd.read_pickle(train_dir)

import pandas as pd



selected_categories = [11, 28,51 ]

additional_data = []  # Initialize an empty list to store the additional data

for category in selected_categories:
    category_data = train_data[train_data['stimulus'] == category]
    existing_count = len(category_data)
    
    if existing_count < 40:
        additional_count = 40 - existing_count
        if additional_count > 0:
            sampled_data = category_data.sample(min(additional_count, len(category_data)), replace=True)
            additional_data.append(sampled_data)

# Concatenate the additional data into a new DataFrame
additional_data_df = pd.concat(additional_data, ignore_index=True)

# Concatenate the original train_data and the additional_data_df
train_data = pd.concat([train_data, additional_data_df], ignore_index=True)

print(train_data['stimulus'].value_counts())  # Display updated category counts


# Shuffle data
train_data = train_data.sample(frac=1).reset_index(drop=True)
eval_data = eval_data.sample(frac=1).reset_index(drop=True)
#train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=21)
# Create train and test datasets
train_dataset = Nina1Dataset(train_data, batch_size=batch_size)
#val_dataset = Nina1Dataset(val_data, batch_size=batch_size)
test_dataset = Nina1Dataset(eval_data, batch_size=batch_size)

print(train_dataset)


0     40
1     40
28    40
29    40
30    40
31    40
32    40
33    40
34    40
35    40
36    40
37    40
38    40
39    40
40    40
41    40
42    40
43    40
44    40
45    40
46    40
47    40
48    40
49    40
50    40
27    40
26    40
25    40
12    40
2     40
3     40
4     40
5     40
6     40
7     40
8     40
9     40
10    40
11    40
13    40
24    40
14    40
15    40
16    40
17    40
18    40
19    40
20    40
21    40
22    40
23    40
51    40
Name: stimulus, dtype: int64


In [10]:
print(train_data['stimulus'])

0        1
1       22
2       51
3        0
4        8
        ..
2075    36
2076    49
2077    29
2078    25
2079     3
Name: stimulus, Length: 2080, dtype: object


In [11]:
print(eval_data['stimulus'])

0        3
1        7
2       21
3       32
4       10
        ..
1035    36
1036    48
1037    28
1038    47
1039    26
Name: stimulus, Length: 1040, dtype: object


In [12]:
from tensorflow.keras.activations import tanh
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dropout, Input, Conv1D, MaxPooling1D,
    Flatten, Dense, ReLU, TimeDistributed, Reshape, BatchNormalization
)
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts
from tensorflow.keras import regularizers, initializers
from keras.regularizers import l2
def cnn(x):
    #print(x.shape)
    #x = Reshape((25, 20,10))(x)  
    x = TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(MaxPooling1D(pool_size=8, strides=2))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=5, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=5, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Conv1D(filters=64, kernel_size=3, padding="same", kernel_initializer="he_normal", strides=2, kernel_regularizer=l2(1e-04)))(x)
    x = TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))(x)
    x = TimeDistributed(Activation('tanh'))(x)
    x = TimeDistributed(Dropout(0.2093))(x)
    x = TimeDistributed(Dropout(0.5))(x)
    #x = TimeDistributed(Activation('relu'))(x)
    x = TimeDistributed(Flatten())(x)
    return x
def Bi_LSTMModel(input_shape,x):
    #model = Sequential()
    # Hidden dimensions
    hidden_dim = 200
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True), input_shape=input_shape)(x)
    x = Dropout(0.3)(x)
    print(x.shape)
    print(11)
    x = Bidirectional(LSTM(hidden_dim, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    print(x.shape)
    print(11)
    #x = Dropout(0.2093)(x)
    x = Flatten()(x)
    # ( ,10000)
    return x
def EMGHandNet(input_shape, num_classes):
    # Define the input layer
    x = Input(shape=input_shape)
    inputs = x
    #print(x.shape)
    #(batch, 25, 20, 10)
    #temp = [cnn(x[:, t, :, :]) for t in range(x.shape[1])]
    #x = tf.stack(temp, axis=1)
    #print(x.shape)
    x = cnn(x)
   
    #print(x.shape)
    x = Bi_LSTMModel(x.shape[1:],x)
    #print(x.shape)
    #x = Dropout(0.2093)(x)
    
    x = Dense(512, activation='tanh')(x)
    #print(x.shape)
    #x = Dropout(0.2093)(x)
    x = BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)(x)
    # Add the output layer
    output_layer = Dense(52, activation='softmax')(x)
    # Create the model
    model = Model(inputs=inputs, outputs=output_layer)
    return model
num_classes = 52
model = EMGHandNet((25, 400, 12), num_classes)
initial_learning_rate = 1e-3
decay_steps = 1000
decay_rate = 0.9
batch_size = 32
# Define your model and its optimizer
# Define learning rate schedule
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)
    # Compile the model with learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
# Define your model and its optimizer
# Define learning rate schedule
with tf.device('/GPU:0'):
    history = model.fit(train_dataset,
                        epochs=200,
                        batch_size=32,
                        validation_data=test_dataset)

(None, 25, 832)
11
(None, 25, 400)
11
(None, 25, 400)
11
Epoch 1/200
65/65 [==============================] - 41s 279ms/step - loss: 4.1712 - accuracy: 0.0692 - val_loss: 5.0981 - val_accuracy: 0.0616
Epoch 2/200
65/65 [==============================] - 15s 234ms/step - loss: 2.9929 - accuracy: 0.1913 - val_loss: 3.1526 - val_accuracy: 0.1951
Epoch 3/200
65/65 [==============================] - 15s 234ms/step - loss: 2.3834 - accuracy: 0.3173 - val_loss: 3.0167 - val_accuracy: 0.2557
Epoch 4/200
65/65 [==============================] - 15s 235ms/step - loss: 1.8244 - accuracy: 0.4712 - val_loss: 3.1314 - val_accuracy: 0.2415
Epoch 5/200
65/65 [==============================] - 16s 249ms/step - loss: 1.4417 - accuracy: 0.5808 - val_loss: 2.8100 - val_accuracy: 0.3068
Epoch 6/200
65/65 [==============================] - 15s 234ms/step - loss: 1.1365 - accuracy: 0.6562 - val_loss: 2.2810 - val_accuracy: 0.4384
Epoch 7/200
65/65 [==============================] - 16s 243ms/step - loss: 0.8

In [13]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25, 400, 12)]     0         
                                                                 
 time_distributed (TimeDistr  (None, 25, 200, 64)      6976      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 25, 200, 64)      256       
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 25, 97, 64)       0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 25, 97, 64)       0         
 tributed)                                                   

In [14]:
true_labels = []  # True labels for the test data
predicted_labels = []  # Predicted labels for the test data

for batch_data, batch_labels in test_dataset:
    batch_predictions = model.predict(batch_data)
    batch_predicted_labels = np.argmax(batch_predictions, axis=1)
    predicted_labels.extend(batch_predicted_labels)
    true_labels.extend(np.argmax(batch_labels, axis=1))

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

accuracy = np.mean(true_labels == predicted_labels)
print(true_labels)
print(predicted_labels)
print(accuracy)

1/1 [==============================] - 0s 34ms/step
[ 3  7 21 ...  0  0  0]
[15  7 21 ...  7  7  7]
0.678030303030303
